In [1]:
import mne
mne.set_log_level("CRITICAL")
import numpy as np
import double_dipper
from double_dipper.io import filePairs, partition
from double_dipper.dataset import mapDataset
from double_dipper.ml import cross_validation

In [2]:
divider = lambda meta: meta["id"]
def labeller(meta):
    strat = meta["strategy"]
    if strat is None: return None
    if strat.lower().startswith("fact"):        return 0
    elif strat.lower().startswith("procedure"): return 1
    else:                                       return None

In [3]:
subjNos = [i for i in range(1,11+1) if i != 5]
pairs = filePairs(*[f"cleaned/main/{i}" for i in subjNos])
dataset = partition(divider, labeller, pairs)

In [4]:
sfreq = double_dipper.constants.sfreq
mapDataset(lambda data: mne.filter.filter_data(data, sfreq, l_freq=1, h_freq=32), dataset)
mapDataset(lambda data: data.reshape([data.shape[0], np.prod(data.shape[1:])]), dataset)

In [5]:
(precision, recall) = cross_validation(dataset)
f1 = 2 * (precision * recall) / (precision + recall)
print(f1)

/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
Held out fold 1: precision = 0.26666666666666666, recall = 0.3076923076923077
/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
Held out fold 2: precision = 0.23076923076923078, recall = 0.2
/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
Held out fold 3: precision = 0.2972972972972973, recall = 0.3793103448275862
/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
Held out fold 4: p

[0.28571429 0.21428571 0.33333333 0.28571429 0.41791045 0.05128205
 0.2745098  0.51020408 0.41791045 0.34666667]


Held out fold 11: precision = 0.2653061224489796, recall = 0.5
